In [1]:
#儲存文章連結 正規表示法吃連結 用來建立連結資料
def getlinkv2 (script):
    global count
    global pkcount
    m = re.findall('(http://udn.com/news/.*\d+)',script)
    udn = 'udn'
    for link in m:
        num = str(pkcount)
        if pkcount < 10:
            num = str(0) * 4 + num
        elif pkcount < 100:     
            num = str(0) * 3 + num
        elif pkcount < 1000:     
            num = str(0) * 2 + num
        elif pkcount < 10000:     
            num = str(0) * 1 + num
        pk = udn + num   
        links = {'link':link,'area':area,'page':page,'count':count,'pk':pk}
        linkary.append(links)
        count +=1
        pkcount +=1

In [2]:
#requests 方法 找出網頁特性 發現是jsp.net 直接對伺服器發送要求
import requests
import re
import time
#列出所需查詢區域 一開始先測試3區的效果
#['中正區','大同區','中山區','松山區','大安區','萬華區','信義區','士林區','北投區','內湖區','南港區','文山區']
ary = ['中正區','大同區','中山區','松山區','大安區','萬華區','信義區','士林區','北投區','內湖區','南港區','文山區']
links = {'link':'','area':'','page':'','count':''}
linkary =[]
#每一區
pkcount = 1
for area in ary:
    count = 1
    headers = {
'Host':'search.udn.com',
'Referer':'http://udn.com/search/result/2/'+ area,
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36'
}
#每一頁    
    for page in range(1,13):
        res =requests.get('http://search.udn.com/search/searchResult4utf8.jsp?ch=udn.news2014&df=2&rc=15&wc=80&pw=220&mc=&q=' + \
                          area + '&fp={}'.format(page),headers = headers,timeout=3)
        time.sleep(3) #避免讓伺服器察覺攻擊
        script = res.text
        getlinkv2(script)

In [3]:
import pandas as pd
data1 = pd.DataFrame(linkary)
data1

,area,count,link,page,pk
0,中正區,1,http://udn.com/news/story/7199/1283073,1,udn00001
1,中正區,2,http://udn.com/news/story/7187/1290064,1,udn00002
2,中正區,3,http://udn.com/news/story/7199/1283069,1,udn00003
3,中正區,4,http://udn.com/news/story/7239/1287644,1,udn00004
4,中正區,5,http://udn.com/news/story/7210/1281408,1,udn00005
5,中正區,6,http://udn.com/news/story/1/1284267,1,udn00006
6,中正區,7,http://udn.com/news/story/2/1283157,1,udn00007
7,中正區,8,http://udn.com/news/story/7192/1278149,1,udn00008
8,中正區,9,http://udn.com/news/story/7328/1278952,1,udn00009
9,中正區,10,http://udn.com/news/story/7266/1278949,1,udn00010


In [4]:
#輸出成CSV檔將連結存起 假如需要可以讀回
data1.to_csv('udn_link.csv',encode='utf-8')

In [5]:
#讀回程式
import csv
linkary = []
with open('udn_link.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        linkary.append(row)

In [6]:
#擷取新聞內文 ptag
def content(pcont):
    cont = ''
    for con in pcont:
        cont = cont + con.text.strip()
    return cont

In [7]:
#取得文章內容及標題
import requests
from bs4 import BeautifulSoup as bs
import pandas
import time

for get in linkary:
    #強制取得 建議不要使用while 在一般requests套件中 會被伺服器端察覺
    while True:
        try:
            url = get['link']
            res = requests.get(url,headers=headers)
            soup = bs(res.content)
            break
        except:
            print 'try again 5sec'
            time.sleep(5)
    for article in soup.select('#story_body_content'):
        try:
            get['title'] = article.select('h2')[0].text.strip()
            pcont = article.select('p')
            get['content'] = content(pcont)
#             get['content'] = article.select('p')[0].text.strip()
            get['date'] = article.select('h3')[0].text.strip().split(' ')[0]
            
            #get['time'] = article.select('h3')[0].text.strip().split(' ')[1] 發文時間可要可不要
        except:
            print 'this link has some problem'
            print get['link'], get['area'], get['count']
            continue

try again 5sec
try again 5sec


c:\python27\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


In [8]:
#將新聞內文轉成 DataFrame
import pandas as pd
data2 = pd.DataFrame(linkary)

#刪除多餘 column
udn_ok = data2.drop(data2.columns[[0]],axis = 1)
udn_ok

,area,content,count,date,link,page,pk,title
0,中正區,營造禪意，取材大自然景物\r\n鈺善閣的創辦人陳健志董事長是高雄旗山農家子弟，因父親的一次意...,1,2015-11-04,http://udn.com/news/story/7199/1283073,1,udn00001,[善導寺站]蔬食養生懷石 精緻料理藝術 鈺善閣
1,中正區,分享 facebook twitter pinterest \r\n當初大家住在...,2,2015-11-04,http://udn.com/news/story/7187/1290064,1,udn00002,懷念美味 眷村媽媽的5大招牌菜是哪些？
2,中正區,複合空間，生活中建立美感\r\n1999 年，從忠孝東路巷弄內「好樣餐廳」（VVG Bist...,3,2015-11-02,http://udn.com/news/story/7199/1283069,1,udn00003,[忠孝新生站]創意無所不在 視覺味覺饗宴 好樣思維
3,中正區,11月1日起，民眾可上聯徵中心網站，申請查閱個人信用報告。 圖／擷自聯徵網站 分享 fa...,4,2015-11-02,http://udn.com/news/story/7239/1287644,1,udn00004,個人信用報告查詢 11月起聯徵網站申請
4,中正區,唯美法式可麗餅位在義二路，不到5坪大的空間，提供義大利麵、燉飯、濃湯、甜點飲料等多樣餐點，開...,5,2015-10-31,http://udn.com/news/story/7210/1281408,1,udn00005,可麗餅放冰淇淋 征服饕客味蕾
5,中正區,同志大遊行今天下午登場，時代力量、綠社盟等第三勢力政黨都率隊參加。綠社盟發出新聞稿指出，若進...,6,2015-10-31,http://udn.com/news/story/1/1284267,1,udn00006,綠社盟推動「跨黨派婚姻平權推動聯盟」
6,中正區,鄭姓男子在北市大同、中正區附近涉嫌用同把鑰匙偷竊三輛機車，台北市警局中正二分局依監視畫面循線...,7,2015-10-30,http://udn.com/news/story/2/1283157,1,udn00007,混血偷車賊 一把鑰匙偷走三輛車
7,中正區,日本拉麵近幾年在台北掀起攻防戰，許多名店如雨後春筍般陸續進軍台北。想品嘗道地日本職人傳授的麵...,8,2015-10-29,http://udn.com/news/story/7192/1278149,1,udn00008,麵Q肉嫩湯濃郁 日本職人超自信美味
8,中正區,基隆的漁港近年因配合政府的漁船減船政策，漁船多轉型為當日或短期近沿海漁業，歷史最悠久、漁產種...,9,2015-10-29,http://udn.com/news/story/7328/1278952,1,udn00009,自豪的大明蝦 漁民自組產銷班賣
9,中正區,衛生局推動各區每年1所學校畫設校際禁菸區計畫，在校園外2公尺畫設紫色「禁菸線」，不讓二手菸進...,10,2015-10-29,http://udn.com/news/story/7266/1278949,1,udn00010,校園禁菸線內 菸蒂不少


In [9]:
#輸出完成品
udn_ok.to_csv('udn.csv',encoding='utf-8')

In [ ]:
# pcont = article.select('p')
# get['content'] = content(pcont)
# def content(pcont):
#     cont = ''
#     for con in pcont:
#         cont = cont + con.text.strip()
#     return cont